# 一、数据预处理

1.图像切割：一般采集到的水样图像会包含盛水容器，并且容器的颜色与水体颜色差异较大，同时位于图像中央，因此截取图像，只保留中央部分。

2.特征提取：颜色特征是一种全局特征，具有稳健性。对切割后的图像提取其颜色矩，作为图像的颜色特征。因为取值范围差别较大，不能直接输入模型（导致模型精度下降），因此在建模之前先标准化数据。

图像数据集：已经分好类别了：类别-编号.jpg

In [88]:
import numpy as np
import os,re    #os 模块用来处理文件路径和目录操作；re 正则表达式模块用来匹配文件名的模式。
from PIL import Image
#PIL 是Python Imaging Library的缩写，它提供了处理图像的功能。
#可以使用导入的 Image 类来打开、操作和处理图像文件。

图像切割及特征提取

In [89]:

path = "C:/Users/liuti/基于水色图像的水质测评/images"  # 图片所在路径

In [90]:
# 自定义获取图片名称函数
def getImgNames(path=path):
    '''
    获取指定路径中所有图片的名称
    :param path: 指定的路径
    :return: 名称列表
    '''
    filenames = os.listdir(path)   # os.listdir() 函数获取指定路径 path 下的所有文件和文件夹的名称
    imgNames = []   #存储符合特定文件名模式的图像文件名称
    for i in filenames:
        if re.findall('^\d_\d+\.jpg$', i) != []:      #文件名符合“数字_数字.jpg结尾的 ；^是行的开始，%是行的结束。
            imgNames.append(i)
            #print(imgNames)  # 添加这一行来打印文件名
    return imgNames

In [91]:
# 批量处理图片数据
imgNames = getImgNames(path=path)  # 获取所有图片名称
n = len(imgNames)        # 图片张数
data = np.zeros([n, 9])  # 用来装样本自变量
'''创建一个二维的NumPy数组 data，其中每个图像的特征将存储在一个行中，
每行有9个元素，用于装载样本自变量。
初始化时，数组中的所有元素都被设置为零。'''
labels = np.zeros([n])   # 用来放样本标签
'''创建一个一维的NumPy数组 labels，用于存储样本标签'''

'创建一个一维的NumPy数组 labels，用于存储样本标签'

In [92]:
n   #共有203张图片

203

地址老是报错：原因是使用 os.path.join 函数可以提高代码的跨平台兼容性，确保代码在不同操作系统上都能正确处理文件路径。这是一个良好的实践，尤其是在处理文件时。但是之前使用的是：img = Image.open(path+imgNames[i])  # 读取图片。PIL库的Image.open函数来打开图像文件，这是一种常见的方法。但是有时候，特别是在处理中文路径或特殊字符的情况下，PIL可能会出现问题。

In [97]:
# 自定义获取三阶颜色矩函数
def Var(data=None):
    '''
    获取给定像素值矩阵的三阶颜色矩
    :param data: 给定的像素值矩阵
    :return: 对应的三阶颜色矩
    '''
    x = np.mean((data-data.mean())**3)   #计算立方差的平均值
    return np.sign(x)*abs(x)**(1/3)   # np.sign() 函数来获取 x 的符号


In [109]:
for i in range(n):
    img_path = os.path.join(path,imgNames[i])  # 读取图片
    #print("Attempting to open file:", img_path)
    img = Image.open(img_path)  # 读取图片
    M,N = img.size  # 图片像素的尺寸:宽、高
    img = img.crop((M/2-50,N/2-50,M/2+50,N/2+50))    # 图片切割
    '''crop 方法接受一个包含左上角和右下角坐标的四元组，表示切割区域。
    在这里，图像被切割为以当前图像中心为中心的 100x100 像素区域。'''
    
    r,g,b = img.split()   #将图片分割成三通道，分别存在红、绿、蓝通道,因此可以分别处理每个颜色通道的数据
    
    rd = np.asarray(r)/255   #(常见)分别将每个颜色通道的图像数据转换为NumPy数组，并将像素值缩放到范围[0, 1]之间。
    gd = np.asarray(g)/255
    bd = np.asarray(b)/255
    
    #颜色矩是一种用于描述图像颜色分布的统计特征
    data[i,0] = rd.mean()  # 一阶颜色矩,代表图像整体明暗程度
    data[i,1] = gd.mean()
    data[i,2] = bd.mean()
    
    data[i,3] = rd.std()   # 二阶颜色矩，代表图像颜色的分布范围
    data[i,4] = gd.std()
    data[i,5] = bd.std()

    data[i,6] = Var(rd)    # 三阶颜色矩，代表图像颜色分布的对称性
    data[i,7] = Var(gd)
    data[i,8] = Var(bd)
    
    labels[i] = imgNames[i][0]  # 从当前图像的文件名提取第一个字符作为样本（类别）标签

模型构建：使用决策树。对标准化之后的样本进行抽样，0.8是训练集，0.2是测试集。

In [110]:
from sklearn.model_selection import train_test_split
# 数据拆分,训练集、测试集
data_tr,data_te,label_tr,label_te = train_test_split(data,labels,test_size=0.2,
                                                     random_state=10)
#random_state=10 是一个随机种子，用于确保每次运行时数据拆分的一致性。


In [111]:

from sklearn.tree import DecisionTreeClassifier
# 模型训练
#.fit(data_tr, label_tr) 使用训练数据来拟合模型(学习如何根据特征数据来进行分类)
model = DecisionTreeClassifier(random_state=5).fit(data_tr, label_tr) 

from sklearn.metrics import confusion_matrix
#使用训练好的决策树分类模型 model 对测试数据 data_te 进行预测
pre_te = model.predict(data_te)


水质评价

In [114]:

# 混淆矩阵_评估分类模型的性能（比较模型的预测结果与实际标签）
cm_te = confusion_matrix(label_te,pre_te)
print(cm_te)


[[ 5  1  5  0]
 [ 3  8  0  0]
 [ 2  0 12  0]
 [ 0  1  0  4]]


每一行代表实际类别，每一列代表预测类别。
第一行表示实际类别0，对应于第一列的值：5个实例被正确分类为类别0，1个实例被误分类为类别1，5个实例被误分类为类别2，没有实例被误分类为类别3。

第二行表示实际类别1，对应于第二列的值：3个实例被正确分类为类别1，8个实例被正确分类为类别1，没有实例被误分类为其他类别。

In [115]:

from sklearn.metrics import accuracy_score
# 准确率
print(accuracy_score(label_te,pre_te))


0.7073170731707317


相对较好